In [9]:
import backtrader as bt
import backtrader.indicators as btind # 导入策略分析模块
import pandas as pd
import tushare as ts
import warnings
import datetime
warnings.filterwarnings("ignore")


In [105]:
def get_ts_data(ts_code, start_date, end_data = None):
    pro = ts.pro_api(token = "64ce1845b91d06f579525db6e53d497b1c513174331f5509320f4bd5")
    df = pro.daily(ts_code=ts_code, start_date=start_date, end_data = end_data)
    df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
    return df.sort_values('trade_date', ascending = True)
    

In [106]:
df = get_ts_data("000001.SZ", "20240401")

In [107]:
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
7,000001.SZ,2024-04-01,10.52,10.65,10.51,10.64,10.52,0.12,1.1407,1191087.96,1261770.380
6,000001.SZ,2024-04-02,10.63,10.68,10.53,10.55,10.64,-0.09,-0.8459,1085458.26,1149700.800
5,000001.SZ,2024-04-03,10.53,10.55,10.42,10.46,10.55,-0.09,-0.8531,981845.85,1028648.883
4,000001.SZ,2024-04-08,10.42,10.49,10.37,10.43,10.46,-0.03,-0.2868,906360.52,945290.441
3,000001.SZ,2024-04-09,10.43,10.47,10.36,10.40,10.43,-0.03,-0.2876,857156.73,891468.991
2,000001.SZ,2024-04-10,10.38,10.41,10.27,10.28,10.40,-0.12,-1.1538,1246390.82,1288696.196
1,000001.SZ,2024-04-11,10.24,10.29,10.12,10.25,10.28,-0.03,-0.2918,1010213.40,1030820.137
0,000001.SZ,2024-04-12,10.22,10.27,10.04,10.06,10.25,-0.19,-1.8537,1305453.71,1322011.754


In [108]:
class TS_Data(bt.feeds.PandasData):
    # 要添加的线 (初始 'close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime')
    lines = ('pre_close', 'change', 'amount', 'extra') 
    # 设置 line 在数据源上的列位置
    # -1表示自动按列明匹配数据
    params = (
        ('fromdate', datetime.datetime(2024,4,1)),
        ('todate', datetime.datetime(2024,4,12)),
        ('nullvalue', 0.0),
        ('dtformat', ('%Y-%m-%d')),
        ('datetime', 1),
        ('open', 2),
        ('high', 3),
        ('low', 4),
        ('close', 5),
        ('pre_close', 6),
        ('change', 7),
        ('openinterest', 8),
        ('volume', 9),
        ('amount', 10),
        ('extra', -1)
    )
    # def _load(self):
    #     # 调用原始的 _load 方法
    #     super()._load()

    #     # 添加你的自定义逻辑
    #     # _load方法是在每一个数据点被加载时调用的，所以你可以在这里添加你的自定义逻辑。
    #     # 你可以访问self.p.dataname来获取原始的Pandas DataFrame，然后从中提取你需要的数据。
    #     self.lines.extra[0] = self.p.dataname['extra'][self._idx]

    #     return True

In [ ]:
class StockCommission(bt.CommInfoBase):
    params = (
                ('stocklike', True), # 指定为股票模式
                ('commtype', bt.CommInfoBase.COMM_PERC), # 使用百分比费用模式
                ('percabs', True), # commission 不以 % 为单位
                ('stamp_duty', 0.001), # 印花税默认为 0.1%
                ('commission', 0.0001), # 交易佣金默认为 0.01%
             ) 
    
    # 自定义费用计算公式
    def _getcommission(self, size, price, pseudoexec):
            if size > 0: # 买入时，只考虑佣金
                return abs(size) * price * self.p.commission
            elif size < 0: # 卖出时，同时考虑佣金和印花税
                return abs(size) * price * (self.p.commission + self.p.stamp_duty)
            else:
                return 0

In [182]:
# 初始化 交易
cerebro = bt.Cerebro()
# 设置初始资金
cerebro.broker.setcash(50000.0)
cerebro.broker.getcash() # 获取当前可用资金

# 设置交易费用
comminfo = StockCommission()                   
cerebro.broker.addcommissioninfo(comminfo)

# 通过调用 brokers 的 set_slippage_perc 方法设置百分比滑点
cerebro.broker.set_slippage_perc(perc=0.0001)


data = TS_Data(dataname=df, fromdate=datetime.datetime(2024,4,2),
                todate=datetime.datetime(2024,4,12), nullvalue = None)
cerebro.adddata(data, name='000001.SZ')

In [183]:
class My_MACD(bt.Indicator):
    lines = ('macd', 'signal', 'histo')
    params = (('period_me1',12),
              ('period_me2', 26),
              ('period_signal', 9),)

    def __init__(self):
        me1 = btind.EMA(self.datas[0].close, period=self.p.period_me1)
        me2 = btind.EMA(self.datas[0].close, period=self.p.period_me2)
        self.l.macd = me1 - me2
        self.l.signal = btind.EMA(self.l.macd, period=self.p.period_signal)
        self.l.histo = self.l.macd - self.l.signal

class TestStrategy(bt.Strategy):
    def __init__(self):
        self.dec = "="*8
        print("{dec} {text:^50s} {dec}".format(dec=self.dec, text="打印 self 策略本身的 lines"))
        print(self.lines.getlinealiases())
        print("{dec} {text:^50s} {dec}".format(dec=self.dec, text="打印 self.datas 第一个数据表格的 lines"))
        print(self.datas[0].lines.getlinealiases())
        print("{dec} {text:^50s} {dec}".format(dec=self.dec, text="打印 最后一天数据"))
        print(bt.num2date(self.datas[0].datetime[0]), end = ':\t')
        print(self.datas[0].close[0], end = "\t")
        print(self.datas[0].openinterest[0])

        ## 
        self.ma3 = btind.SimpleMovingAverage(self.datas[0].close, period=3)
        self.ma5 = btind.SimpleMovingAverage(self.datas[0].close, period=5)
        self.macd = My_MACD(period_me1=3,period_me2=5,period_signal=2)
        
    def next(self):
        print("{dec} {text:^50s} {dec}".format(dec=self.dec, text="打印 当天数据"))
        print(bt.num2date(self.datas[0].datetime[0]), self.ma3[0], self.ma5[0], 
              self.macd.macd[0], self.macd.signal[0], self.macd.histo[0])
        if self.ma5 > self.ma3:
            print("买买买")
        

In [184]:
cerebro.addstrategy(TestStrategy)
cerebro.run()

========                打印 self 策略本身的 lines                 ========
('datetime',)
========            打印 self.datas 第一个数据表格的 lines            ========
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'pre_close', 'change', 'amount', 'extra')
========                     打印 最后一天数据                      ========
2024-04-12 00:00:00:	10.06	-1.8537
========                      打印 当天数据                       ========
2024-04-11 00:00:00 10.31 10.364 -0.06100000000000172 -0.06250000000000178 0.0015000000000000568
买买买
========                      打印 当天数据                       ========
2024-04-12 00:00:00 10.196666666666667 10.284 -0.0815000000000019 -0.07516666666666853 -0.006333333333333371
买买买


In [171]:
print?

Signature: print(*args, sep=' ', end='\n', file=None, flush=False)
Docstring:
Prints the values to a stream, or to sys.stdout by default.

sep
  string inserted between values, default a space.
end
  string appended after the last value, default a newline.
file
  a file-like object (stream); defaults to the current sys.stdout.
flush
  whether to forcibly flush the stream.
Type:      builtin_function_or_method